<a href="https://colab.research.google.com/github/SH0123/DeepLearning/blob/master/logisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [0]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [0]:
x_data = torch.FloatTensor([[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]])
y_data = torch.FloatTensor([[0], [0], [0], [1], [1], [1]])

model = BinaryClassifier()

optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    hypothesis = model(x_data)

    cost = F.binary_cross_entropy(hypothesis, y_data)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_data
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100
        ))

Epoch    0/100 Cost: 0.778947 Accuracy 33.33%
Epoch   10/100 Cost: 0.606802 Accuracy 66.67%
Epoch   20/100 Cost: 0.446548 Accuracy 66.67%
Epoch   30/100 Cost: 0.376169 Accuracy 83.33%
Epoch   40/100 Cost: 0.318945 Accuracy 83.33%
Epoch   50/100 Cost: 0.268428 Accuracy 83.33%
Epoch   60/100 Cost: 0.222594 Accuracy 100.00%
Epoch   70/100 Cost: 0.183695 Accuracy 100.00%
Epoch   80/100 Cost: 0.158160 Accuracy 100.00%
Epoch   90/100 Cost: 0.144616 Accuracy 100.00%
Epoch  100/100 Cost: 0.134716 Accuracy 100.00%


In [0]:
#Cross Entropy Loss(low level)

z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)

#[3,]
y = torch.randint(5, (3,)).long()

y_one_hot = torch.zeros_like(hypothesis)
#[3, 5]
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

#[3, 1]
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1)
"""
F.nll_loss(F.log_softmax(z, dim=1), y)로 나타낼 수 있음
F.cross_entropy(z, y)가 가장 간단
"""
#scalar
cost = cost.mean()

print(hypothesis)
print(y)
print(y_one_hot)
print(cost)

tensor([[0.2528, 0.1500, 0.2019, 0.1292, 0.2662],
        [0.1518, 0.2729, 0.1887, 0.1441, 0.2425],
        [0.1761, 0.2659, 0.1987, 0.2173, 0.1420]], grad_fn=<SoftmaxBackward>)
tensor([0, 3, 0])
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.]])
tensor(1.6830, grad_fn=<MeanBackward0>)


In [0]:
#Training Cross Entropy Loss(high level)

class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)
    
    def forward(self, x):
        return self.linear(x)

x_train = torch.FloatTensor([[1, 2, 1, 1],
                            [2, 1, 3, 2],
                            [3, 1, 3, 4],
                            [4, 1, 5, 5],
                            [1, 7, 5, 5],
                            [1, 2, 5, 6],
                            [1, 6, 6, 6],
                            [1, 7, 7, 7]])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    pred = model(x_train)
    cost = F.cross_entropy(pred, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.193624
Epoch  100/1000 Cost: 0.647261
Epoch  200/1000 Cost: 0.559104
Epoch  300/1000 Cost: 0.503264
Epoch  400/1000 Cost: 0.458217
Epoch  500/1000 Cost: 0.418269
Epoch  600/1000 Cost: 0.380891
Epoch  700/1000 Cost: 0.344497
Epoch  800/1000 Cost: 0.307935
Epoch  900/1000 Cost: 0.271172
Epoch 1000/1000 Cost: 0.243551


multi layer perceptron을 통해 XOR 문제 해결

In [12]:
x = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]])
y = torch.FloatTensor([[0], [1], [1], [0]])

class MultiPercep(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(2, 2, bias=True)
        self.linear2 = nn.Linear(2, 1, bias=True)
        self.sigmoid = nn.Sigmoid()
        self.percep = nn.Sequential(self.linear1, self.sigmoid, self.linear2, self.sigmoid)

    def forward(self, input):
        output = self.percep(input)

        return output

criterion = nn.BCELoss()
model = MultiPercep()
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):
    
    hypothesis = model(x)
    cost = criterion(hypothesis, y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(epoch, cost.item())



0 0.728512167930603
1000 0.023579902946949005
2000 0.006828591227531433
3000 0.003952228929847479
4000 0.00277394475415349
5000 0.002134685404598713
6000 0.0017338610487058759
7000 0.0014592853840440512
8000 0.0012595225125551224
9000 0.0011076723458245397
10000 0.0009883784223347902
